# La clase DataSet

Si queremos empezar a entrenar algoritmos lo primero que necesitamos son datos. La mayoría de las veces tendremos los datos guardados en archivos externos en diferentes formatos. Como inversión para el futuro vamos a crear una forma fácil de cargar y manipular datos para que posteriormente los algoritmos de nuestra librería `HappyML` los usen.

En este primer notebook vamos a crear una clase llamada `DataSet` que se encargará de encapsular nuestros datos.

##### Un poco de matemáticas

En primer lugar definiremos matemáticamente qué es un **dataset** $\mathcal{D}$.

$$
\mathcal{D} = (\mathbf{x}_1, \mathbf{y}_1), (\mathbf{x}_2, \mathbf{y}_2), \dots, (\mathbf{x}_N, \mathbf{y}_N)
$$

en el que el $\mathbf{x}_i$ es el vector de entrada del ejemplo $i$ y $\mathbf{y}_i$ es el vector de salida asociado al ejemplo $i$. Dichos vectores se definen como

$$
\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_d \end{bmatrix} \in \mathbb{R}^d
, \qquad
\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_k \end{bmatrix} \in \mathbb{R}^k
.
$$

Podemos simplificar el concepto de dataset si juntamos los datos de entrada en una matriz y los de salida en otra.

$$
\mathbf{X} = \begin{bmatrix}
- & \mathbf{x}_1^\mathrm{T} & - \\
- & \mathbf{x}_2^\mathrm{T} & - \\
  & \vdots       &   \\
- & \mathbf{x}_N^\mathrm{T} & - \\
\end{bmatrix} \in \mathbb{R}^{N \times d}
, \qquad
\mathbf{Y} = \begin{bmatrix}
- & \mathbf{y}_1^\mathrm{T} & - \\
- & \mathbf{y}_2^\mathrm{T} & - \\
  & \vdots       &   \\
- & \mathbf{y}_N^\mathrm{T} & - \\
\end{bmatrix} \in \mathbb{R}^{N \times k}
$$

## Implementando clase DataSet

En nuestra implementación almacenaremos los datos usando matrices. Esta forma de hacerlo mejora la eficiencia de cualquier operación que queramos realizar con los datos. Las operaciones matriciales son altamente paralelizables. `numpy` se encargará de paralelizar los cálculos por nosotros simplificándonos mucho la vida. A partir de ahora, siempre que podamos usaremos operaciones con matrices en lugar de realizar bucles.

In [2]:

import numpy as np


class DataSet():
    """Generic collection of inputs and outputs.
    
    """

    X = np.empty((0, 0))

    Y = np.empty((0, 0))


    def get_N(self):
        """Gets the number of samples in the dataset.
        
        """
        # The next two expressions are not necessarily equivalent:
        # self.X.shape[0] and self.Y.shape[0]
        # self.Y.shape[0] <-- Can be 0 if no output assigned.
        return self.X.shape[0]

    def get_d(self):
        """Gets the dimension of each sample in the dataset.
        
        """
        return self.X.shape[1]

    def get_k(self):
        """Gets the number of outputs of each sample.
        
        """
        return self.Y.shape[1]


## Leyendo ficheros

Si creamos un objeto de la clase `DataSet` contendrá dos matrices vacías. Vamos a añadir un par de métodos auxiliares que nos permitan rellenar esas matrices.

In [1]:

def load(filename, delimiter="", n_outputs=1, one_shot_output=False, header=False):
    # Set delimiters if filename has a know extension.
    if delimiter is "":
        if filename.endswith(".csv"):
            delimiter = ","
        else:
            delimiter = None
    # Open file and load dataset from stream.
    return load_from_stream(open(filename), delimiter=delimiter, n_outputs=n_outputs,
                            one_shot_output=one_shot_output, header=header)


def load_from_stream(stream, delimiter=",", n_outputs=1,
                     one_shot_output=False, header=False):
    # Check parameters.
    assert not (one_shot_output and abs(n_outputs) != 1), \
        "If one-shot output is selected the number of outputs must be 1."
    # Read stream.
    data = np.loadtxt(stream, delimiter=delimiter, skiprows=int(header))
    # Check feature dimensions.
    d = data.shape[1]
    assert d >= abs(n_outputs), \
        "Number of outputs greater than number of data columns."
    # Set starts/ends of the submatrixes X and Y.
    if n_outputs <= 0:
        start_X = 0
        end_X = start_Y = d + n_outputs
        end_Y = d
    else:
        start_Y = 0
        end_Y = start_X = n_outputs
        end_X = d
    # Create DataSet object.
    dataset = DataSet()
    dataset.X = data[:, start_X:end_X]
    dataset.Y = data[:, start_Y:end_Y]
    if one_shot_output:
        max_output = dataset.Y.max()
        min_output = dataset.Y.min()
        N = dataset.get_N()
        k = max_output - min_output + 1
        indexes = np.add(dataset.Y, -min_output)
        indexes = indexes.astype(int).reshape(N)
        dataset.Y = np.zeros((N, k))
        dataset.Y[np.arange(0, N), indexes] = 1

    return dataset


## Pruebas

Lo primero que debemos de hacer es crear un archivo con un dataset de pruebas. Comprobamos que está correctamente guardado y mostramos su contenido. Como puede observarse está en formato CSV.

In [5]:
filename = "../resources/dataset01.csv"
str = open(filename).read()
print str

1,1,1
1,-1,1
-1,1,-1
-1,-1,-1



Vamos a cargarlo usando nuestro método `load`. Por defecto se toma la primera columna como si fuera el valor de salida $\mathbf{y}$.

In [16]:
dataset = load(filename)

print "X:\n", dataset.X
print "Y:\n", dataset.Y
print "N:", dataset.get_N()
print "d:", dataset.get_d()
print "k:", dataset.get_k()

X:
[[ 1.  1.]
 [ 1. -1.]
 [-1.  1.]
 [-1. -1.]]
Y:
[[ 1.]
 [ 1.]
 [-1.]
 [-1.]]
N: 4
d: 2
k: 1


In [8]:
dataset = load(filename, n_outputs=2)

print "N:", dataset.get_N()
print "d:", dataset.get_d()
print "k:", dataset.get_k()

N: 4
d: 1
k: 2


In [14]:
dataset = load(filename, n_outputs=0)

print "N:", dataset.get_N()
print "d:", dataset.get_d()
print "k:", dataset.get_k()

N: 4
d: 3
k: 0


## Guardando DataSets en archivos

De nuevo con ayuda de *numpy* vamos a crear una función que nos guarde un objeto `DataSet` en el disco.

In [3]:

def save(file, dataset, delimiter=",", header="", footer=""):
    data = np.column_stack((dataset.Y, dataset.X))
    np.savetxt(file, data, delimiter=delimiter, header=header, footer=footer)


Si por algún motivo se deseara imprimir por la terminal el valor del `DataSet` bastaría con llamar a la función `save` con `sys.stdout` como primer parámetro.

In [7]:

import sys

filename = "../resources/dataset01.csv"
dataset = load(filename)
save(sys.stdout, dataset)


1.000000000000000000e+00,1.000000000000000000e+00,1.000000000000000000e+00
1.000000000000000000e+00,-1.000000000000000000e+00,1.000000000000000000e+00
-1.000000000000000000e+00,1.000000000000000000e+00,-1.000000000000000000e+00
-1.000000000000000000e+00,-1.000000000000000000e+00,-1.000000000000000000e+00
